In [1]:
import pandas as pd
from auxiliary.paths import input_path, raw_path
from astropy.table import Table
import numpy as np
from auxiliary.columns import wise, galex, create_colors, calculate_colors, splus, error_splus
from auxiliary.correct_extinction import correction
import numpy as np

In [2]:
filename = "/Volumes/GoogleDrive/My Drive/Research/Projects/qucats_predict/data/input_model/STRIPE82-0170_dual_VAC_features.fits"
table = Table.read(os.path.join(input_path, filename))
table = table.to_pandas()
table["ID"] = table["ID"].str.decode('utf-8') 

In [3]:
table['W1_MAG'] = 22.5 - 2.5*np.log10(table['FW1']) # + 2.699
table['W2_MAG'] = 22.5 - 2.5*np.log10(table['FW2']) # + 3.339

# http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/namespacelsst_1_1afw_1_1image.html#a0a023f269211d52086723788764d484e
# return std::abs(fluxErr / (-0.4 * flux * std::log(10)));
table['e_W1_MAG'] = abs(table['e_FW1'] / (-0.4*table['FW1']*np.log(10)))
table['e_W2_MAG'] = abs(table['e_FW2'] / (-0.4*table['FW2']*np.log(10)))

table['W1_MAG'].replace([np.inf, -np.inf], np.nan, inplace=True)
table['W2_MAG'].replace([np.inf, -np.inf], np.nan, inplace=True)

/Users/Mariko/opt/anaconda3/envs/photoz/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/Mariko/opt/anaconda3/envs/photoz/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [4]:
table[wise+galex] = table[wise+galex].fillna(value=99)

In [5]:
correct = False

if correct:
    table = correction(table)
calculate_colors(table, broad=True, narrow=True, wise=True, galex = True, aper="PStotal")
features = create_colors(broad=True, narrow=True, wise=True, galex=True, aper="PStotal")
pos_splus = ["ID", "RA", "DEC"]

if correct:
    table[pos_splus+features+splus+wise+galex+error_splus].to_csv("/Volumes/GoogleDrive/My Drive/Research/Projects/qucats_predict/data/input_model/STRIPE82-0170_QSOz_VAC_ext.csv", index=False)
else:
    table[pos_splus+features+splus+wise+galex+error_splus].to_csv("/Volumes/GoogleDrive/My Drive/Research/Projects/qucats_predict/data/input_model/STRIPE82-0170_QSOz_VAC.csv", index=False)

In [12]:
corrected_df.dropna(subset=splus)
corrected_df.query(f"ID=='iDR4_3_STRIPE82-0170_0022523'")[splus]

,u_PStotal,g_PStotal,r_PStotal,i_PStotal,z_PStotal,J0378_PStotal,J0395_PStotal,J0410_PStotal,J0430_PStotal,J0515_PStotal,J0660_PStotal,J0861_PStotal
22522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
features

['u_PStotal-r_PStotal',
 'g_PStotal-r_PStotal',
 'r_PStotal-i_PStotal',
 'r_PStotal-z_PStotal',
 'J0378_PStotal-r_PStotal',
 'J0395_PStotal-r_PStotal',
 'J0410_PStotal-r_PStotal',
 'J0430_PStotal-r_PStotal',
 'J0515_PStotal-r_PStotal',
 'r_PStotal-J0660_PStotal',
 'r_PStotal-J0861_PStotal',
 'r_PStotal-W1_MAG',
 'r_PStotal-W2_MAG',
 'FUVmag-r_PStotal',
 'NUVmag-r_PStotal']

In [12]:
Magnitudes

['FUVmag',
 'NUVmag',
 'u_PStotal',
 'g_PStotal',
 'r_PStotal',
 'i_PStotal',
 'z_PStotal',
 'J0378_PStotal',
 'J0395_PStotal',
 'J0410_PStotal',
 'J0430_PStotal',
 'J0515_PStotal',
 'J0660_PStotal',
 'J0861_PStotal',
 'W1_MAG',
 'W2_MAG']

In [17]:
features

['u_PStotal-r_PStotal',
 'g_PStotal-r_PStotal',
 'r_PStotal-i_PStotal',
 'r_PStotal-z_PStotal',
 'J0378_PStotal-r_PStotal',
 'J0395_PStotal-r_PStotal',
 'J0410_PStotal-r_PStotal',
 'J0430_PStotal-r_PStotal',
 'J0515_PStotal-r_PStotal',
 'r_PStotal-J0660_PStotal',
 'r_PStotal-J0861_PStotal',
 'r_PStotal-W1_MAG',
 'r_PStotal-W2_MAG',
 'FUVmag-r_PStotal',
 'NUVmag-r_PStotal']

In [15]:
Dataframe = table.copy(deep=True)
Magnitudes = galex+splus+wise
Aper = "PStotal"
Reference_Mag  = 'r_'+Aper
Reference_Idx = Magnitudes.index(Reference_Mag)
MagnitudesToLeft = Magnitudes[:Reference_Idx]
MagnitudesToRight = Magnitudes[(Reference_Idx+1):]

for feature in MagnitudesToLeft: # of Reference_Mag
    Dataframe[feature+'-'+Reference_Mag] = Dataframe[feature] - Dataframe[Reference_Mag]

for feature in MagnitudesToRight: # of Reference_Mag
    Dataframe[Reference_Mag+'-'+feature] = Dataframe[Reference_Mag] - Dataframe[feature]

Training_Features = []
for s in Dataframe.columns.values:
    if '-' in s:
        if s.split('-')[0] in Magnitudes and s.split('-')[1] in Magnitudes:
            Training_Features.append(s)

In [16]:
Dataframe.columns.values

array(['ID', 'PHOT_ID_dual_1', 'RA', 'DEC', 'A', 'B_in', 'KRON_RADIUS',
       'FWHM_n', 'MU_MAX_INST', 'ISOarea', 'SEX_FLAGS_DET',
       's2n_DET_aper_6', 'PHOT_ID_dual_2', 'SEX_FLAGS_u', 'u_iso',
       'e_u_iso', 'u_aper_3', 'e_u_aper_3', 'u_aper_6', 'e_u_aper_6',
       'u_auto', 'e_u_auto', 'u_petro', 'e_u_petro', 'u_PStotal',
       'e_u_PStotal', 'FWHM_n_u', 'MU_MAX_u', 'PHOT_ID_dual_3',
       'SEX_FLAGS_J0378', 'J0378_iso', 'e_J0378_iso', 'J0378_aper_3',
       'e_J0378_aper_3', 'J0378_aper_6', 'e_J0378_aper_6', 'J0378_auto',
       'e_J0378_auto', 'J0378_petro', 'e_J0378_petro', 'J0378_PStotal',
       'e_J0378_PStotal', 'FWHM_n_J0378', 'MU_MAX_J0378',
       'PHOT_ID_dual_4', 'SEX_FLAGS_J0395', 'J0395_iso', 'e_J0395_iso',
       'J0395_aper_3', 'e_J0395_aper_3', 'J0395_aper_6', 'e_J0395_aper_6',
       'J0395_auto', 'e_J0395_auto', 'J0395_petro', 'e_J0395_petro',
       'J0395_PStotal', 'e_J0395_PStotal', 'FWHM_n_J0395', 'MU_MAX_J0395',
       'PHOT_ID_dual_5', 'SEX_FLAGS_J